# Gaussova eliminacija
---

## Općenito

Sustav $Ax=b$
se rješava u tri koraka (__bez pivotiranja__):

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija),
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija),
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija).

Ukoliko pivotiramo, tada je

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.407875  0.00391919  0.952309  0.553738  0.597253  0.214889
 0.207402  0.00839688  0.188561  0.856547  0.273671  0.736088
 0.277228  0.899913    0.698302  0.484642  0.284409  0.729688
 0.956715  0.809309    0.958642  0.663639  0.680984  0.362086
 0.623493  0.733428    0.85406   0.977112  0.49684   0.146818
 0.877573  0.379       0.958652  0.579757  0.412833  0.247733

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
 1.0         0.0     0.0       0.0       0.0      0.0
 0.508494    1.0     0.0       0.0       0.0      0.0
 0.679688  140.108   1.0       0.0       0.0      0.0
 2.34561   124.94    0.859905  1.0       0.0      0.0
 1.52864   113.591   0.795238  0.365716  1.0      0.0
 2.15157    57.8651  0.38621   0.85388   3.85513  1.0,

6x6 Array{Float64,2}:
 0.407875  0.00391919   0.952309    0.553738   0.597253     0.214889
 0.0       0.006404    -0.295682    0.574975  -0.0300283    0.626819
 0.0       0.0         41.4784    -80.4501     4.08566    -87.2385  
 0.0       0.0          0.0        -3.29317   -0.481478    -3.43994 
 0.0       0.0          0.0         0.0       -0.078184    -0.749272
 0.0       0.0          0.0         0.0        0.0          3.03269 )

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0          0.0  0.0           0.0          0.0           0.0        
 0.0          0.0  0.0           0.0          0.0           0.0        
 0.0          0.0  2.44249e-15   6.21725e-15  2.22045e-16  -4.21885e-15
 1.11022e-16  0.0  1.55431e-15  -1.08802e-14  2.22045e-16   1.06581e-14
 0.0          0.0  5.10703e-15   2.22045e-15  0.0          -5.55112e-15
 1.11022e-16  0.0  8.88178e-16  -8.88178e-16  0.0          -4.44089e-15

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.457505
 0.523093
 0.987595
 0.535645
 0.586494
 0.417633

In [7]:
# Rijesimo susatav pomocu ugradjene funkcije:
x=A\b

6-element Array{Float64,1}:
 -0.472074
  0.309572
  0.581203
  0.185441
 -0.211651
  0.554137

In [8]:
# Rijesimo sustav pomocu nasih funkcija
y=myL(L,b)

6-element Array{Float64,1}:
   0.457505
   0.290454
 -40.0183  
  -2.41498 
  -0.398652
   1.68053 

In [9]:
x1=myU(U,y)

6-element Array{Float64,1}:
 -0.472074
  0.309572
  0.581203
  0.185441
 -0.211651
  0.554137

In [10]:
# Usporedimo rjesenja
x-x1

6-element Array{Float64,1}:
  1.83187e-15
  3.88578e-15
  4.44089e-16
 -7.29972e-15
  2.30371e-15
  7.10543e-15

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice i ne računa s blok matricama.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1 (vidi [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [11]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [12]:
mylu1(A)

6x6 Array{Float64,2}:
 0.407875    0.00391919   0.952309    0.553738   0.597253     0.214889
 0.508494    0.006404    -0.295682    0.574975  -0.0300283    0.626819
 0.679688  140.108       41.4784    -80.4501     4.08566    -87.2385  
 2.34561   124.94         0.859905   -3.29317   -0.481478    -3.43994 
 1.52864   113.591        0.795238    0.365716  -0.078184    -0.749272
 2.15157    57.8651       0.38621     0.85388    3.85513      3.03269 

In [13]:
L,U

(
6x6 Array{Float64,2}:
 1.0         0.0     0.0       0.0       0.0      0.0
 0.508494    1.0     0.0       0.0       0.0      0.0
 0.679688  140.108   1.0       0.0       0.0      0.0
 2.34561   124.94    0.859905  1.0       0.0      0.0
 1.52864   113.591   0.795238  0.365716  1.0      0.0
 2.15157    57.8651  0.38621   0.85388   3.85513  1.0,

6x6 Array{Float64,2}:
 0.407875  0.00391919   0.952309    0.553738   0.597253     0.214889
 0.0       0.006404    -0.295682    0.574975  -0.0300283    0.626819
 0.0       0.0         41.4784    -80.4501     4.08566    -87.2385  
 0.0       0.0          0.0        -3.29317   -0.481478    -3.43994 
 0.0       0.0          0.0         0.0       -0.078184    -0.749272
 0.0       0.0          0.0         0.0        0.0          3.03269 )

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [14]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.2516     0.191832   0.842952   …  0.374108   0.2304     0.646028 
 0.28979    0.36691    0.371066      0.790459   0.811175   0.606334 
 0.0459577  0.204151   0.0338093     0.465089   0.598063   0.0977611
 0.750103   0.0161151  0.165072      0.647708   0.373439   0.991384 
 0.380244   0.360456   0.849208      0.833002   0.725416   0.0904813
 0.0891538  0.535158   0.877206   …  0.0909253  0.345141   0.586253 
 0.685598   0.251301   0.30524       0.152832   0.334099   0.667848 
 0.659047   0.754688   0.732688      0.690562   0.900245   0.465177 
 0.724867   0.598233   0.95503       0.733141   0.69062    0.33355  
 0.469034   0.252997   0.192736      0.0292803  0.430327   0.469653 
 0.684823   0.595011   0.272188   …  0.643129   0.0831563  0.155625 
 0.478494   0.525652   0.23624       0.70623    0.845384   0.497098 
 0.620253   0.322442   0.845636      0.584924   0.982135   0.188027 
 ⋮                                ⋱             ⋮                   
 0.84611

In [16]:
@time lu(A);

  0.187391 seconds (55 allocations: 6.010 MB)


In [18]:
@time mylu1(A);

  0.383187 seconds (17.26 k allocations: 1.004 GB, 8.63% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji od `lu()`. `mylu1()` je malo sporiji od `lu()`.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [19]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [20]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [21]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.084012  0.93731   0.293078  0.371178 
 0.949653  0.685362  0.882906  0.0994496
 0.962597  0.925026  0.2404    0.0123154
 0.390697  0.497566  0.486494  0.535175                           …  4x4 Array{Float64,2}:
 0.0486935  0.425281  0.351231  0.256995
 0.217795   0.20976   0.822085  0.168398
 0.905219   0.529935  0.471659  0.95463 
 0.779906   0.127414  0.632476  0.378935                    
 4x4 Array{Float64,2}:
  0.032512   -0.193367  0.429998   0.686491
  0.0275252  -0.809537  0.670505   1.7922  
 -0.776783    0.205559  0.0149778  1.53059 
 -0.440224   -1.33219   1.07926    2.32193                  4x4 Array{Float64,2}:
 -0.548339  0.482542    0.139838   -0.237628 
 -1.74937   0.282703   -0.0705115  -0.523164 
 -0.530076  0.0884251  -0.503984   -0.0185226
 -1.78794   0.200865    0.134915   -1.00068  
 4x4 Array{Float64,2}:
 -0.633159   -0.889003   0.375102    2.18428  
  0.286766    0.298826  -0.0243404   0.190864 
  1.00028     0.694276 

In [22]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [23]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                          …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                        
 4x4 Array{Float64,2}:
 0.0          0.0          0.0           0.0        
 1.11022e-16  0.0          1.11022e-16  -2.22045e-16
 0.0          1.11022e-16  1.11022e-16  -2.22045e-16
 1.11022e-16  1.11022e-16  4.44089e-16   0.0                 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                        
 4x4 Array{Float64,2}:
 0.0          0.0   1.11022e-16   0.0        
 5.55112e-17  0.0  -5.55112e-17  -5.55112e-17
 1.59595e-16  0.0   0.0          

In [24]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [25]:
norm(unblock(Res))

8.784958395936023e-16

Sada probajmo veću dimenziju:

In [26]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [27]:
@time mylu2(Ab);

  0.081171 seconds (8.79 k allocations: 26.526 MB, 5.77% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> 
> što u praksi dovoljno spriječava rast elemenata.


In [28]:
A=rand(5,5)
L,U,P=lu(A)

(
5x5 Array{Float64,2}:
 1.0        0.0        0.0        0.0        0.0
 0.455445   1.0        0.0        0.0        0.0
 0.332549   0.350739   1.0        0.0        0.0
 0.953197  -0.962446  -0.279472   1.0        0.0
 0.667283  -0.570325  -0.0843338  0.0826613  1.0,

5x5 Array{Float64,2}:
 0.945959  0.370105  0.432166   0.978507   0.196637
 0.0       0.342623  0.102859  -0.302302   0.822171
 0.0       0.0       0.321609  -0.0744019  0.152325
 0.0       0.0       0.0       -0.926844   1.18687 
 0.0       0.0       0.0        0.0        0.627599,

[1,2,3,5,4])

In [29]:
L*U-A[P,:]

5x5 Array{Float64,2}:
 0.0          0.0  0.0   0.0          0.0        
 0.0          0.0  0.0   0.0          0.0        
 0.0          0.0  0.0   0.0          0.0        
 1.11022e-16  0.0  0.0  -1.11022e-16  1.11022e-16
 0.0          0.0  0.0   0.0          1.11022e-16

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [30]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [31]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.372761
 0.12211 
 0.579845
 0.667646
 0.777744

In [32]:
L,U,Pr,Pc=gecp(A)

(
5x5 Array{Float64,2}:
 1.0         0.0        0.0        0.0       0.0
 0.0498763   1.0        0.0        0.0       0.0
 0.49061     0.100543   1.0        0.0       0.0
 0.0128666   0.239697   0.0234988  1.0       0.0
 0.784974   -0.214723  -0.0554173  0.575376  1.0,

5x5 Array{Float64,2}:
 0.983708  0.27753   0.461596  0.510677  0.138958  
 0.0       0.890938  0.711844  0.615641  0.637985  
 0.0       0.0       0.682045  0.234998  0.529192  
 0.0       0.0       0.0       0.612271  0.225779  
 0.0       0.0       0.0       0.0       0.00218769,

5x5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0,

5x5 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0)

In [33]:
Pr*L*U*Pc'-A

5x5 Array{Float64,2}:
 0.0  0.0   0.0          0.0          0.0
 0.0  0.0   0.0          0.0          0.0
 0.0  0.0   0.0          2.77556e-17  0.0
 0.0  0.0   0.0          0.0          0.0
 0.0  0.0  -1.11022e-16  5.55112e-17  0.0

In [34]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.667646 
  0.0888104
  0.0362774
  0.747014 
 -0.352973 

In [35]:
z=myU(U,y)

5-element Array{Float64,1}:
  -54.2732 
   -9.66901
  104.319  
   60.7173 
 -161.345  

In [36]:
x=Pc*z

5-element Array{Float64,1}:
   -9.66901
  -54.2732 
   60.7173 
 -161.345  
  104.319  

In [37]:
A*x-b

5-element Array{Float64,1}:
  5.9952e-15 
 -1.42109e-14
  9.99201e-15
  4.21885e-15
  1.75415e-14

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$

za neki $\hat x=x+\delta x$.

Želimo ocijeniti 

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$

odnosno

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$, imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je __uvjetovanost__ (__kondicija__)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [Numeričke matematike, str. 42][RS04]:


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [38]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [39]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [40]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [41]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [42]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [43]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [44]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [Matrix Computations, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [45]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.358789
 0.931848
 0.574445
 0.486609
 0.717028
 0.235816
 0.151405
 0.219889
 0.690114
 0.408721

In [51]:
# Vandermonmde-oove matrice imaju veliku kondiciju.
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0          1.0       1.0         …  1.0          1.0        1.0        
 0.358789     0.931848  0.574445       0.219889     0.690114   0.408721   
 0.12873      0.868341  0.329987       0.0483514    0.476257   0.167053   
 0.0461868    0.809161  0.18956        0.010632     0.328672   0.0682782  
 0.0165713    0.754015  0.108892       0.00233786   0.226821   0.0279068  
 0.00594562   0.702628  0.0625523   …  0.00051407   0.156532   0.0114061  
 0.00213322   0.654742  0.0359329      0.000113039  0.108025   0.00466191 
 0.000765378  0.61012   0.0206415      2.4856e-5    0.0745496  0.00190542 
 0.000274609  0.568539  0.0118574      5.46557e-6   0.0514477  0.000778787
 9.85269e-5   0.529792  0.00681143     1.20182e-6   0.0355048  0.000318307

In [52]:
b=rand(n)

10-element Array{Float64,1}:
 0.480947 
 0.850592 
 0.0937389
 0.856666 
 0.499379 
 0.293559 
 0.853992 
 0.484377 
 0.656302 
 0.328297 

In [53]:
x=A\b

10-element Array{Float64,1}:
    -1.53966e7
 -3756.92     
     4.71478e6
    -1.14531e7
     1.4306e6 
     1.50358e7
     5.65474e5
    -1.24705e7
    -2.51152e6
     2.00888e7

In [54]:
cond(A)

3.8254277601125824e8

In [55]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
 -1.539662225749555787755272658215960142754520428936196302680433935251035627437854e+07
 -3.756923026284087771981360130199430474029643017529217530542680528021848999984731e+03
  4.714781624478642149029375518234507221797516030486818483816124353216226658773933e+06
 -1.145307112831204465096728008868738511666530485538537138861188557133832954278736e+07
  1.430602746757213368256515124007905170329913618840183918555990354799304595765219e+06
  1.503582495567228240506773471515540156055929994097531732851584863586229590020181e+07
  5.654738055274437047542708625201140494504338703991703473839793907682464121160014e+05
 -1.247047367028444915369601753688846677520605538194528511149585801040879961685568e+07
 -2.511522178683646625960071794240639151540314760824441768665211503434148396425707e+06
  2.008876350631389421189017042342332106065323443391391098171089438357358211259037e+07

In [56]:
norm(xb-x)/norm(xb)

1.780399100548250681495721628097032940747616614043211988638411386262549293497005e-10

### Umjetno loša kondicija

In [57]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)

([-1.0,2.0],50000.00017991671,[8.881784197001252e-16,-4.440892098500626e-16])

## Rezidual


Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava (vidi [Afternotes on Numerical Analysis, str. 128][Ste96]):


$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

__Rezidual__ (ili __ostatak__) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$

pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,

> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
> 
> ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [58]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [59]:
norm(r)/(norm(A)*norm(x))

0.0